# LIMPIEZA TABLA VENTAS Y CARGA A SILVER

In [0]:
from pyspark.sql.functions import col, to_date, regexp_replace, current_timestamp, trim, try_to_date

# 1. Cargar desde Bronze
df_bronze = spark.read.table("workspace.bronze.ventas")

# Función para limpiar decimales latinos (7,14 -> 7.14)
def fix_decimal(column_name):
    return regexp_replace(
        regexp_replace(col(column_name), r'\.', ''), 
        r',', '.'
    ).cast("double")

# 2. Procesamiento Silver
df_silver_ventas = df_bronze.select(
    # SOLUCIÓN: Usamos try_to_date para que ignore los textos "Caja", "Reporte", etc.
    try_to_date(trim(col("fecha")), "dd/MM/yyyy").alias("fecha"),
    col("cliente").cast("string"),
    col("producto").cast("string"),
    fix_decimal("cantidad").cast("int").alias("cantidad"),
    # ID como String
    col("numero_de_identificacion").cast("string").alias("numero_identificacion"),
    col("cod_producto").alias("codigo_producto"),
    # Precios y Totales
    fix_decimal("precio_unitario").alias("precio_unitario"),
    fix_decimal("subtotal").alias("subtotal"),
    fix_decimal("iva").alias("iva"),
    fix_decimal("total").alias("total")
).filter(
    # Solo filas que realmente tengan una fecha válida y un total
    (col("fecha").isNotNull()) & 
    (col("total").isNotNull())
).withColumn("fecha_carga_silver", current_timestamp())

# 3. Guardar en Silver
df_silver_ventas.write.mode("overwrite").option("overwriteSchema", "true").saveAsTable("workspace.silver.ventas")

print("✅ Tabla 'Ventas' procesada exitosamente en Capa Silver.")